In [97]:
import numpy as np
import torch
from torch.utils import data
from torch.nn.functional import normalize
#from d2l import torch as d2l

In [98]:
T=normalize(torch.FloatTensor(np.loadtxt("T.txt")),dim=0)
r_cross=normalize(torch.FloatTensor(np.loadtxt("r_cross.txt")),dim=0)
denorm=torch.norm(torch.FloatTensor(np.loadtxt("r_cross.txt")),p=2,dim=0)

class MyDataset(data.Dataset):
    def __init__(self, feature_data, label_data):
        self.feature_data = feature_data
        self.label_data = label_data

    def __len__(self):
        return len(self.label_data)

    def __getitem__(self, idx):
        return self.feature_data[idx], self.label_data[idx]

dataset = MyDataset(T,r_cross)
train_size=int(0.8*len(r_cross))
test_size=len(r_cross)-train_size

data_train, data_test = data.random_split(dataset, [train_size, test_size])

In [99]:
batch_size = 80
data_iter = data.DataLoader(data_train, batch_size, shuffle=True)
test_data_iter = data.DataLoader(data_test, batch_size, shuffle=False)

In [100]:
next(iter(data_iter))


[tensor([[-0.0708, -0.0684, -0.0671,  ..., -0.0652, -0.0666, -0.0638],
         [-0.0719, -0.0715, -0.0710,  ..., -0.0718, -0.0719, -0.0740],
         [-0.0727, -0.0739, -0.0730,  ..., -0.0727, -0.0723, -0.0750],
         ...,
         [-0.0721, -0.0719, -0.0715,  ..., -0.0724, -0.0726, -0.0750],
         [-0.0720, -0.0736, -0.0748,  ..., -0.0721, -0.0708, -0.0701],
         [-0.0709, -0.0701, -0.0702,  ..., -0.0712, -0.0719, -0.0740]]),
 tensor([0.0429, 0.0827, 0.0759, 0.0643, 0.0947, 0.0752, 0.0548, 0.0749, 0.0347,
         0.0449, 0.0729, 0.0504, 0.0732, 0.0368, 0.0456, 0.0507, 0.0562, 0.0885,
         0.0609, 0.0858, 0.0402, 0.0555, 0.0650, 0.0640, 0.0688, 0.0681, 0.0538,
         0.0460, 0.0912, 0.0909, 0.0817, 0.0701, 0.0569, 0.0831, 0.0977, 0.0735,
         0.0746, 0.0552, 0.0769, 0.0783, 0.0453, 0.0902, 0.0780, 0.1018, 0.0725,
         0.0432, 0.0623, 0.0483, 0.0664, 0.0357, 0.0923, 0.0756, 0.0511, 0.0929,
         0.0586, 0.0824, 0.0684, 0.0994, 0.0626, 0.0678, 0.0381, 0.0606,

In [101]:
net=torch.nn.Sequential(torch.nn.Linear(128,128),torch.nn.ReLU(),torch.nn.Linear(128,128),torch.nn.ReLU(),torch.nn.Linear(128,1))

In [111]:
loss = torch.nn.MSELoss()
l = loss(net(T).squeeze(-1), r_cross)
need_l=6e-6
T_test, r_cross_test=next(iter(test_data_iter))
l_test = loss(net(T_test).squeeze(-1), r_cross_test)

while l_test>need_l:

    for i in (0,2,4):
        net[i].weight.data.normal_(0, 0.15)
        net[i].bias.data.fill_(0)

    trainer = torch.optim.SGD(net.parameters(), lr=0.03)

    num_epochs = 200
    for epoch in range(num_epochs):
        for X, y in data_iter:
            l = loss(net(X).squeeze(-1) ,y)
            trainer.zero_grad()
            l.backward()
            trainer.step()
        l = loss(net(T).squeeze(-1), r_cross)
        if l <= need_l:
            break
    print(f'epoch {epoch + 1}, loss {l:f}')
    l_test = loss(net(T_test).squeeze(-1), r_cross_test)
    print(f"test_loss {l_test:f}")

print(net(T_test).squeeze(-1)*denorm*1e4)
print(r_cross_test*denorm*1e4)


epoch 200, loss 0.000010
test_loss 0.000010
epoch 200, loss 0.000048
test_loss 0.000049
epoch 200, loss 0.000009
test_loss 0.000011
epoch 200, loss 0.000012
test_loss 0.000013
epoch 200, loss 0.000012
test_loss 0.000012
epoch 200, loss 0.000011
test_loss 0.000012
epoch 200, loss 0.000016
test_loss 0.000019
epoch 200, loss 0.000052
test_loss 0.000055
epoch 200, loss 0.000044
test_loss 0.000045
epoch 200, loss 0.000025
test_loss 0.000028
epoch 200, loss 0.000012
test_loss 0.000012
epoch 200, loss 0.000018
test_loss 0.000021
epoch 200, loss 0.000028
test_loss 0.000032
epoch 200, loss 0.000119
test_loss 0.000127
epoch 118, loss 0.000006
test_loss 0.000007
epoch 200, loss 0.000011
test_loss 0.000011
epoch 200, loss 0.000042
test_loss 0.000040
epoch 200, loss 0.000015
test_loss 0.000017
epoch 87, loss 0.000006
test_loss 0.000007
epoch 54, loss 0.000006
test_loss 0.000008
epoch 200, loss 0.000062
test_loss 0.000069
epoch 200, loss 0.000011
test_loss 0.000013
epoch 200, loss 0.000123
test_loss